In [1]:
from tkinter import *
from functools import partial
import numpy as np
import pandas as pd
import json
from PIL import ImageTk, Image
import os

In [4]:
# replace with your own names/logos


teamnames= ['gen', 'gen', 'gen', 'gen', 'gen', 'gen', 'gen', 'gen']
teamnicknames = ['NN1', 'NN2', 'NN3', 'NN4', 'NN5', 'NN6', 'NN7', 'NN8']
teamlogos=['gen.jpeg','gen.jpeg', 'gen.jpeg', 'gen.jpeg', 'gen.jpeg', 'gen.jpeg', 'gen.jpeg', 'gen.jpeg']
typecode = {'Batsmen':'BT','Wicketkeepers':'WK','All-Rounders':'AR','Pacers':'BP','Spinners':'BS'}

In [4]:


df=pd.read_excel('IPLAuction2021.xlsx')


In [5]:
sizes=[(160,160),(160,160),(128,200),(160,208),(160,196),(144,201),(160,196),(144,212)]
ownpics = [Image.open('IPLaucpics/{}.jpeg'.format(teamnames[i])).resize(sizes[i], Image.ANTIALIAS) for i in range(8)]
logos=[Image.open('IPLaucpics/{}logo.jpeg'.format(i)).resize((125,125),Image.ANTIALIAS) for i in teamnames]
plpics = {i[:-4]:Image.open('IPLaucpics/Players/{}'.format(i)) for i in os.listdir('IPLaucpics/Players')}

In [6]:
def displaysquad(num):
    clearsquad()
    plyr = buyslists[teamnames[num]]
    for h in range(len(plyr)):
        squad.insert(h+1,plyr[h])
    teamowner.config(text = teamnicknames[num],font=('Helvetica',12,'bold'),image = ownpicsf[num], compound ='top')    

def clearsquad():
    squad.delete(0,18)
    teamowner.config(text='',image='')
    
def clearsq(event):
    clearsquad()
    
def squaddata(event,num):
    clearsquad()
    person = teamnames[num]
    teamowner.config(text = teamnicknames[num],font=('Helvetica',12,'bold'),image = ownpicsf[num], compound ='top')    
    for a,b in [(1,'Batsmen'),(4,'Wicketkeepers'),(7,'All-Rounders'),(10,'Pacers'),(13,'Spinners')]:
        squad.insert(a, b.upper())
        squad.insert(a+1,teamdata.loc[person,typecode[b]])
        squad.insert(a+2,' ')
    

In [7]:
def nextpl(event):
    if processed:
        nextplayer()


def nextplayer():
    global bidder
    global price
    global bidnum
    global foreign
    global plno
    global processed
    plno+=1
    bidder = []
    price = 0
    bidnum = 0
    foreign = False
    
    sold.config(text='')
    currentbidder.config(text='')
    currentprice.config(text='')
    
    global ser
    
    ser = df.iloc[plno-1]
    if ~ser.isnull().any():
        plinfo.config(text = '{0}\n\nTeam: {1}\n\nCountry:{2}'.format(*ser.values))            
        player.config(image=plpicsf.get(str(plno+1),plpicsf['gen']))
        bsprice.config(text = 'Base Price: {}L'.format(int(ser[3])))
        processed = False
        price = int(ser[3])
        if ser[2] != 'India':
            foreign = True
    else:
        categ.config(text=ser[0])
        plinfo.config(text='')
        player.config(image='')
        bsprice.config(text='')
        
def bidding(num):
    
    global bidder
    global price
    global bidnum
    if processed:
        print('error')
    elif price != 0:
        bidder.append(teamnames[num])
        currentbidder.config(text = 'Current Bidder: '+bidder[-1])
        bidnum += 1
        if bidnum > 1:
            if price < 200:
                price += 10
            elif price < 1000:
                price += 20
            else:
                price += 25
        
        currentprice.config(text = "Current Bid: " + str(price)+ 'L')
    
def revers(event):
    global bidder
    global price
    global bidnum
    global processed
    try:
        currentbidder.config(text ="Current Bidder: " +bidder[-2])
    except:
        currentbidder.config(text = '')
    bidder.pop()    
    bidnum -= 1
    if price == ser[3]:
        pass
    elif price <= 200:
        price -= 10
    elif price <= 1000:
        price -= 20
    else:
        price -= 25
        
    currentprice.config(text = "Current Bid: " + str(price)+ 'L')   
    
    
    
    
    
def soldd(event):
    global bidder
    global price
    global bidnum
    global foreign
    global processed
    if bidnum != 0:
        buyer = bidder[-1]
        sold.config(text = 'SOLD!\n\nTo: {}\n\nFor: {} Lakhs'.format(buyer, price))
        buyslists[buyer].append(ser[0])
        
        ind = teamnames.index(buyer)+1
        balance = float(purs[ind]['text'][:-2]) - 0.01*price
        purs[ind].config(text = '%.2f'%balance+'Cr')
        tot[ind].config(text=int(tot[ind]['text'])-1)
        teamdata.loc[buyer,'Total']+=1
        teamdata.loc[buyer,ser[4]]+=1
        if foreign:
            ovrses[ind].config(text=int(ovrses[ind]['text'])-1)
            teamdata.loc[buyer,'Overseas'] += 1
        teamdata.loc[buyer,'Purse Spent']+=price*0.01
        soldlist=soldlist.append(ser)
        soldlist['TeamName']=buyer
        soldlist['Price'] = price
    else:
        sold.config(text = 'UNSOLD!')
        unsoldlist=unsoldlist.append(ser)
    processed = True
    
    

In [18]:
buyslists = {teamnames[i]:[] for i in range(8)}
unsoldlist = pd.DataFrame(columns = df.columns)
soldlist = pd.DataFrame(columns=df.columns)
teamdata = pd.DataFrame(columns=['Total','Overseas','WK','BT','BS','BP','AR','Purse Spent'],index=teamnames)
teamdata.fillna(0, inplace=True)


In [11]:
### DAY2 , (If required) 
#teamsheet=pd.read_excel('Auction2021.xlsx',sheet_name='Teamsheet')
#teamdata=pd.read_excel('Auction2021.xlsx',sheet_name='Teamdata',index_col=0)
#buyslists = {i:[j for j in teamsheet[i].values if j != ' '] for i in teamsheet.columns}

In [19]:
bidder = []
bidnum = 0
foreign = False
price = 0
processed= True

In [ ]:
scr = Tk()
scr.geometry('1000x750')
scr.title("Auction")
#########################
plpicsf = {i:ImageTk.PhotoImage(plpics[i]) for i in plpics.keys()}
logopics = {l:ImageTk.PhotoImage(logos[l]) for l in range(8)}

teams1 = Frame(scr, bg = 'grey')
teams1.place(relx = 0.02, rely = 0.56, relwidth = 0.55, relheight = 0.21)
teams2 = Frame(scr, bg = 'grey')
teams2.place(relx = 0.02, rely = 0.77, relwidth = 0.55, relheight = 0.21)

categ = Label(scr, bg = 'yellow')
categ.place(relx = 0.02, rely = 0.02, relwidth = 0.25, relheight = 0.05)

player = Label(scr, bg = 'grey', font =(20))
player.place(relx = 0.02, rely = 0.08, relwidth = 0.25, relheight = 0.3)

plinfo = Label(scr, bg = 'red',fg='white',font = ('Arial',10,'bold'))
plinfo.place(relx = 0.02, rely = 0.39, relwidth = 0.25, relheight = 0.16)

bsprice = Label(scr, bg = 'navy',fg='white', font = (20))
bsprice.place(relx = 0.32, rely = 0.02, relwidth = 0.23, relheight = 0.07)


currentprice = Label(scr, bg = 'navy',fg='white', font = (20))
currentprice.place(relx = 0.32, rely = 0.1, relwidth = 0.23, relheight = 0.1) 

currentbidder = Label(scr, bg = 'navy',fg='white', font = (20))
currentbidder.place(relx = 0.32, rely = 0.21, relwidth = 0.23, relheight = 0.1) 


sold = Label(scr, bg = 'white', font = (20) )
sold.place(relx = 0.32, rely = 0.32, relwidth = 0.23, relheight= 0.23)
###############################################################
flexiframe = Frame(scr, bg = 'white')
flexiframe.place(relx=0.6, rely = 0.55, relwidth = 0.35, relheight = 0.43)

ffbutt = Frame(flexiframe)
ffbutt.place(relx=0,rely=0,relheight = 0.1,relwidth=1)
ffbutts = {}
for po in range(8):
    ffbutts[po]=Button(ffbutt,bg='yellow', text=str(po+1), command = lambda k = po:displaysquad(k))
    ffbutts[po].pack(side=LEFT,fill=BOTH,expand=True)
    

squad = Listbox(flexiframe,bg='navy',height=18,fg='white',font=('Helvetica',13))
squad.place(relx=0, rely=0.1,relwidth=0.42, relheight=0.9)

teamowner = Label(flexiframe,bg='orange red')
teamowner.place(relx=0.42, rely=0.1, relheight=0.9,relwidth=0.58)

ownpicsf = [ImageTk.PhotoImage(p) for p in ownpics]
#######################################################
dataframe = Frame(scr, bg = 'grey')
dataframe.place(relx = 0.6, rely = 0.02, relwidth = 0.35, relheight=0.5)

teamnam = [Label(dataframe, fg='white',bg='navy',font = ('Times New Roman', 15), anchor='w',relief='ridge') for i in range(10)]
tot = [Label(dataframe, fg='white',bg='navy',font = ('Times New Roman', 15)) for i in range(10)]
ovrses = [Label(dataframe, fg='white',bg='navy',font = ('Times New Roman', 15)) for i in range(10)]
purs = [Label(dataframe, fg='black',bg='turquoise2',font = ('Times New Roman', 15), anchor='e',relief='raised') for i in range(10)]

for i in range(9):
    teamnam[i].place(relx=0, rely=0+0.11*i,relwidth=0.57,relheight=0.1)
    tot[i].place(relx=0.57, rely=0+0.11*i,relwidth=0.12,relheight=0.1)
    ovrses[i].place(relx=0.69, rely=0+0.11*i,relwidth=0.11,relheight=0.1)
    purs[i].place(relx=0.8, rely=0+0.11*i,relwidth=0.2,relheight=0.1)
    
teamnam[0].config(text='Team',bg='navy',fg='Darkgoldenrod3',font = ('Times New Roman', 15,'bold'))
tot[0].config(text="Tot",bg='navy',fg='Darkgoldenrod3',font = ('Times New Roman', 15,'bold'))
ovrses[0].config(text="Ovr",bg='navy',fg='Darkgoldenrod3',font = ('Times New Roman', 15,'bold'))
purs[0].config(text='Purse',bg='navy',fg='Darkgoldenrod3',font = ('Times New Roman', 15,'bold'))

for m in range(1,9):
    teamnam[m].config(text=teamnicknames[m-1])
    teamnam[m].bind('<Button 1>',lambda x=None,y=m-1:squaddata(x,y))
    tot[m].config(text=18-teamdata.loc[teamnames[m-1],'Total'])
    ovrses[m].config(text=7-teamdata.loc[teamnames[m-1],'Overseas'])
    purs[m].config(text='{:.2f}Cr'.format(85.00-teamdata.loc[teamnames[m-1],'Purse Spent']))
###########################
teamlist = {}
for i in range(4):
    teamlist[i] = Button(teams1,image =logopics[i],bg = 'slate grey', font = (40), command = lambda k = i : bidding(k))
    teamlist[i].pack(expand=True, fill=BOTH, side = LEFT)
for j in range(4,8):
    teamlist[j] = Button(teams2,image = logopics[j],bg = 'slate grey', font = (40), command= lambda k = j : bidding(k))
    teamlist[j].pack(expand=True, fill=BOTH,side = LEFT)



    
    
    
    
    
scr.bind('<Right>',nextpl)
scr.bind('<Return>', soldd)
scr.bind('<Left>',revers)
scr.bind('<c>',clearsq)
scr.bind('<s>',warn)'
scr.mainloop()

In [30]:

with pd.ExcelWriter('Auction2021.xlsx') as exl:
    teamsheet.to_excel(exl,sheet_name='Teamsheet',na_rep=' ',index=False)
    unsoldlist.to_excel(exl,sheet_name='Unsold')
    soldlist.to_excel(exl,sheet_name='Sold')
    teamdata.to_excel(exl,sheet_name='Teamdata')


In [ ]:
'''
with pd.ExcelWriter('Auction2021.xlsx') as exl:
    teamsheet.to_excel(exl,sheet_name='Teamsheet',na_rep=' ',index=False)
    #unsoldlist.to_excel(exl,sheet_name='Unsold')
    #soldlist.to_excel(exl,sheet_name='Sold')
    teamdata.to_excel(exl,sheet_name='Teamdata')'''